In [16]:
! pip3 install --upgrade pip
! pip install -U langchain openai psycopg2-binary langchain_experimental langchain-openai

In [1]:
from langchain import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
import os
import csv
import psycopg2

In [2]:
DB_USERNAME = os.environ.get("DB_USERNAME")
DB_PASSWORD = os.environ.get("DB_PASSWORD")
DB_HOST = os.environ.get("DB_HOST")
DB_PORT = os.environ.get("DB_PORT")
DB_NAME = os.environ.get("DB_NAME")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
GPT_MODEL = "gpt-3.5-turbo-0125"

## Setup Database
https://blog.devart.com/configure-postgresql-to-allow-remote-connection.html

```bash
sudo apt install postgresql
sudo -i -u postgres
createdb <db name>
createuser --interactive --pwprompt <username>
psql
GRANT ALL PRIVILEGES ON DATABASE <db name> TO <username>;
\q
exit
sudo nano /etc/postgresql/12/main/pg_hba.conf
sudo systemctl restart postgresql
psql -U <username> -d <db name>
```

## Create tables
```sql
CREATE TABLE travel_category (
    category_id INTEGER PRIMARY KEY,
    main_category VARCHAR(100) NOT NULL,
    category VARCHAR(100) NOT NULL,
    sub_category VARCHAR(100) NOT NULL,
    emission_factor DOUBLE PRECISION NOT NULL,
    transaction_unit VARCHAR(100) NOT NULL
);
```
```sql
CREATE TABLE employee (
    employee_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    department VARCHAR(100) NOT NULL
);
```
```sql
CREATE TABLE travel (
    travel_id SERIAL PRIMARY KEY,
    source VARCHAR(100) NOT NULL,
    destination VARCHAR(100) NOT NULL,
    transaction_value DOUBLE PRECISION NOT NULL,
    cost DOUBLE PRECISION NOT NULL,
    date DATE NOT NULL,
    category_id INTEGER REFERENCES travel_category(category_id) ON DELETE CASCADE,
    employee_id INTEGER REFERENCES employee(employee_id) ON DELETE CASCADE,
    transaction_unit VARCHAR(100) NOT NULL
);
```
```sql
CREATE TABLE date (
    date_id VARCHAR(100) PRIMARY KEY,
    quarter INTEGER NOT NULL,
    month INTEGER NOT NULL,
    year INTEGER NOT NULL
);
```

## Load the data

In [3]:
conn = psycopg2.connect(
    dbname=DB_NAME, user=DB_USERNAME, password=DB_PASSWORD, host=DB_HOST
)

cursor = conn.cursor()

In [6]:
def create_table(table_query):
    cursor.execute(table_query)
    conn.commit()
    print(f"Table creation successfully.")


def load_data(csv_file, table_name):
    with open(csv_file, "r", newline="") as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            try:
                placeholders = ",".join(["%s"] * len(row))
                sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
                cursor.execute(sql, row)
            except psycopg2.Error as e:
                conn.rollback()
                print(f"Error inserting row: {e}")
                continue
    conn.commit()
    print(f"Data loaded into {table_name} table successfully.")

In [7]:
create_travel_category_table_query = """
    CREATE TABLE travel_category (
        category_id INTEGER PRIMARY KEY,
        main_category VARCHAR(100) NOT NULL,
        category VARCHAR(100) NOT NULL,
        sub_category VARCHAR(100) NOT NULL,
        emission_factor DOUBLE PRECISION NOT NULL,
        transaction_unit VARCHAR(100) NOT NULL
    );
"""

create_employee_table_query = """
    CREATE TABLE employee (
        employee_id INTEGER PRIMARY KEY,
        title VARCHAR(100) NOT NULL,
        department VARCHAR(100) NOT NULL
    );
"""

create_travel_table_query = """
    CREATE TABLE travel (
        travel_id SERIAL PRIMARY KEY,
        source VARCHAR(100) NOT NULL,
        destination VARCHAR(100) NOT NULL,
        transaction_value DOUBLE PRECISION NOT NULL,
        cost DOUBLE PRECISION NOT NULL,
        date DATE NOT NULL,
        category_id INTEGER REFERENCES travel_category(category_id) ON DELETE CASCADE,
        employee_id INTEGER REFERENCES employee(employee_id) ON DELETE CASCADE,
        transaction_unit VARCHAR(100) NOT NULL
    );
"""

In [8]:
create_table(create_travel_category_table_query)
create_table(create_employee_table_query)
create_table(create_travel_table_query)

load_data("employee.csv", "employee")
load_data("travel_category.csv", "travel_category")
load_data("travel.csv", "travel")

Table creation successfully.
Table creation successfully.
Table creation successfully.
Data loaded into employee table successfully.
Data loaded into travel_category table successfully.
Data loaded into travel table successfully.


In [9]:
cursor.close()
conn.close()

## Setup LLM

In [10]:
pg_uri = (
    f"postgresql+psycopg2://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)
db = SQLDatabase.from_uri(pg_uri)
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name=GPT_MODEL)
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True, top_k=3)

/Users/noah/anaconda3/lib/python3.11/site-packages/langchain_experimental/sql/base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [11]:
PROMPT = """ 
Given an input question, first create a syntactically correct Postgresql query without ```sql run,  
then look at the results of the query, inerpret th results and return an answer.  
The question: {question}
"""

## Run a query

In [12]:
question = "what is the average carbo emission for 2023?"
# response = db_chain.run(question) # instead if you don't have a prompt
response = db_chain.run(PROMPT.format(question=question))

/Users/noah/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
 
Given an input question, first create a syntactically correct Postgresql query without ```sql run,  
then look at the results of the query and return the answer.  
The question: what is the average carbo emission for 2023?

SQLQuery:SELECT AVG(tc.emission_factor) AS average_emission
FROM travel_category tc
JOIN travel t ON tc.category_id = t.category_id
WHERE t.date >= '2023-01-01' AND t.date <= '2023-12-31';
SQLResult: [(0.4009302325581428,)]
Answer:The average carbon emission for 2023 is approximately 0.4009.
> Finished chain.


In [13]:
response

'The average carbon emission for 2023 is approximately 0.4009.'